# Solving Linear Optimisation Problems with Mosek Fusion

This is an explanation of the linear optimisation code using Mosek Fusion. We will run through two linear optimisation cases, one solvable the other not. 

As this is a tutorial introduction we first introduce the linear problem we wish to solve.

\begin{split}\begin{array} {lccccccccl}
\mbox{Maximize}    & 3 x_0  & +  & 1 x_1   & +  & 5 x_2  & +  & 1 x_3 &       &  \\
\mbox{Subject to}  & 3 x_0  & +  & 1 x_1   & +  & 2 x_2  &    &       & =     & 30,  \\
                   & 2 x_0  & +  & 1 x_1   & +  & 3 x_2  & +  & 1 x_3 & \geq  & 15,  \\
                   &          &    & 2 x_1 &     &       & +  & 3 x_3 & \leq  & 25,
\label{solvable}
\end{array}\end{split}

With bounds 
\begin{split}\begin{array}{ccccc}
0  & \leq  & x_0  & \leq  & \infty ,  \\
0  & \leq  & x_1  & \leq  & 10,  \\
0  & \leq  & x_2  & \leq  & \infty ,\\
0  & \leq  & x_3  & \leq  & \infty .
\end{array}\end{split}

We structure the code as follows. Our optimisation code will all live in one python function - main. main takes two arguments, matrix A of linear constraints (technically a 2D array) and vector B of linear objective function to maximise. We hardcode the constraints into main, although this is something we might change in future 

Inside function main we invoke class Model, which contains the optimisation problem. In the sequel we refer constantly to Fusion classes which are helpfully listed at https://docs.mosek.com/8.1/pythonfusion/class_index.html

First we must import Fusion.

In [2]:
from mosek.fusion import *

Now we enter the arguments for our function main.

In [3]:
A = [[3.0, 1.0, 2.0, 0.0],
     [2.0, 1.0, 3.0, 1.0],
     [0.0, 2.0, 0.0, 3.0]]
c = [3.0, 1.0, 5.0, 1.0]

Before we start in earnest let us reflect on what we seek. In solving our problem we seek a point in 4-space which maximises the given functional. Our code will need to 
1. Initialise a 4-vector $\textbf{x}$ in which the solution is stored
2. Express constraints on the domain of $\textbf{x}$. There are global constraints (constraints uniformly applied to each  component of x) and local constraints (constraints applied to only one component of x)
3. Solve the model and output the data

 Now we tell Mosek we are implementing a model by envoking the model class. 

In [11]:
with Model("SL") as SL:
     x = SL.variable("x", 4, Domain.greaterThan(0.0))
     SL.constraint(x.index(1), Domain.lessThan(10.0))
    
     SL.constraint("c1", Expr.dot(A[0], x), Domain.equalsTo(30.0))
     SL.constraint("c2", Expr.dot(A[1], x), Domain.greaterThan(15.0))
     SL.constraint("c3", Expr.dot(A[2], x), Domain.lessThan(25.0))
        
     SL.objective("obj", ObjectiveSense.Maximize, Expr.dot(c, x))
    
     SL.solve()
     sol=x.level()

In the second line, first block we have initialised a solution vector $\textbf{x}$ with the global constraint (the L.H.S of our bounds) using commands from the $\textbf{Domain}$ class.

For upper bounds we only have one non-infinite bound - on $\{\textbf{x}\}_1$. We therefore need a way to set local bounds. This is achieved in the third line, first block. 
The other variables are unbounded thus the absense of any commands for them.

In the second block we implement the linear constraints we have inputted through array A. We symbolically inner product A and x using commands from the $\textit{Expr}$ class, and set with domain class commands we have seen before. 
$\textit{Note all the constraints live in the constraint attribute of the model (here SL)}$

In the final block we set the objective function - the one we seek to maximise/minimise via similar commands

All that remains is to output the data, which can be done conjunctely via list comprehensions.

In [12]:
print('\n'.join(["x[%d] = %f" % (i, sol[i]) for i in range(4)]))

x[0] = 0.000000
x[1] = 0.000000
x[2] = 15.000000
x[3] = 8.333333


NB. Notice x is not a variable but a solution class with its own attributes. We output the solution to sol using the level suffix.

# An Unsolvable Problem

\begin{split}\begin{array} {lccccccccl}
\mbox{Minimize}    & 2 x_0  & +  & 3 x_1   & -  & 1 x_2         &  \\
\mbox{Subject to}  & 1 x_0  & +  &  x_1   & +  & 2 x_2  =     & 1,  \\
                   & -2 x_0  & -  & x_1   & +  & 1 x_2   & =  & -0.5,  \\
                   & -  x_0  &    &         & +  & 5 x_3   &=   & -0.1 ,
\label{unsolvable}
\end{array}\end{split}

In [26]:
from mosek.fusion import *
B = [[1.0, 1.0, 2.0],
     [-2.0, -1.0, 1.0],
     [-1.0, 0.0, 5.0]]
d = [2.0, 3.0, -1.0]

  
with Model("unsolvable") as LU:

    y = LU.variable("y", 3, Domain.greaterThan(0.0))
        
    LU.constraint("c1", Expr.dot(B[0], y), Domain.equalsTo(1.0))
    LU.constraint("c2", Expr.dot(B[1], y), Domain.equalsTo(-0.5))
    LU.constraint("c3", Expr.dot(B[2], y), Domain.equalsTo(-0.1))

    # Set the objective function 
    LU.objective("obj", ObjectiveSense.Maximize, Expr.dot(c, y))

    # Solve the problem
    LU.solve()
    sol=x.level()

AttributeError: _Model__solutionptr

ie some probems have been thrown up.